**I would like to thank @code1110. This notebook is inspired from him his work!**

https://www.kaggle.com/code1110/janestreet-faster-inference-by-xgb-with-treelite

# Install treelite

In [ ]:
!pip --quiet install ../input/treelite/treelite-0.93-py3-none-manylinux2010_x86_64.whl

In [ ]:
!pip --quiet install ../input/treelite/treelite_runtime-0.93-py3-none-manylinux2010_x86_64.whl

# Imports 

In [ ]:
import numpy as np
import pandas as pd

import os, sys
import gc
import math
import random
import pathlib
from tqdm import tqdm
from typing import List, NoReturn, Union, Tuple, Optional, Text, Generic, Callable, Dict
from sklearn.preprocessing import MinMaxScaler, StandardScaler, QuantileTransformer
from sklearn.decomposition import PCA
from sklearn import linear_model
import operator
import xgboost as xgb
import lightgbm as lgb

# treelite
import treelite
import treelite_runtime 

sns.set_context("talk")
style.use('fivethirtyeight')
pd.options.display.max_columns = None

import warnings
warnings.filterwarnings('ignore')

## PurgedGroupTimeSeriesSplit

In [ ]:
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

# Denoising the Target feature

In [ ]:
# Snippets from Marco Lopez de Prado, 2020

from scipy.optimize import minimize
from sklearn.neighbors import KernelDensity

def mpPDF(var,q,pts):
    # Marcenko-Pastur pdf
    # q=T/N
    eMin, eMax = var*(1-(1./q)**.5)**2, var*(1+(1./q)**.5)**2
    eVal = np.linspace(eMin,eMax,pts)
    pdf = q/(2*np.pi*var*eVal)*((eMax-eVal)*(eVal-eMin))**.5
    pdf = pd.Series(pdf.reshape(-1,), index=eVal.reshape(-1,))
    return pdf


def getPCA(matrix):
    # Get eVal,eVec from a Hermitian matrix
    eVal,eVec = np.linalg.eigh(matrix)
    indices=eVal.argsort()[::-1] # arguments for sorting eVal desc
    eVal,eVec=eVal[indices],eVec[:,indices]
    eVal=np.diagflat(eVal)
    return eVal,eVec

def fitKDE(obs,bWidth=.25,kernel='gaussian',x=None):
    # Fit kernel to a series of obs, and derive the prob of obs
    # x is the array of values on which the fit KDE will be evaluated
    if len(obs.shape)==1:
        obs=obs.reshape(-1,1)
    kde=KernelDensity(kernel=kernel,bandwidth=bWidth).fit(obs)
    if x is None:
        x=np.unique(obs).reshape(-1,)
    if len(x.shape)==1:
        x=x.reshape(-1,1)
    logProb=kde.score_samples(x) # log(density)
    pdf=pd.Series(np.exp(logProb),index=x.flatten())
    return pdf

def cov2corr(cov):
    # Derive the correlation matrix from a covariance matrix
    std=np.sqrt(np.diag(cov))
    corr=cov/np.outer(std,std)
    corr[corr<-1],corr[corr>1]=-1,1 # numerical error
    return corr

def errPDFs(var,eVal,q,bWidth,pts=1000):
    # Fit error
    pdf0=mpPDF(var,q,pts) # theoretical pdf
    pdf1=fitKDE(eVal,bWidth,x=pdf0.index.values) # empirical pdf
    sse=np.sum((pdf1-pdf0)**2)
    return sse

def findMaxEval(eVal,q,bWidth):
    # Find max random eVal by fitting Marcenko’s dist
    out=minimize(lambda *x:errPDFs(*x),.5,args=(eVal,q,bWidth),
    bounds=((1E-5,1-1E-5),))
    if out['success']:
        var=out['x'][0]
    else:
        var=1
    eMax=var*(1+(1./q)**.5)**2
    return eMax,var

def denoisedCorr(eVal,eVec,nFacts):
    # Remove noise from corr by fixing random eigenvalues
    eVal_=np.diag(eVal).copy()
    eVal_[nFacts:]=eVal_[nFacts:].sum()/float(eVal_.shape[0] - nFacts)
    eVal_=np.diag(eVal_)
    corr1=np.dot(eVec,eVal_).dot(eVec.T)
    corr1=cov2corr(corr1)
    return corr1

def denoisedCorr2(eVal,eVec,nFacts,alpha=0):
    # Remove noise from corr through targeted shrinkage
    eValL,eVecL=eVal[:nFacts,:nFacts],eVec[:,:nFacts]
    eValR,eVecR=eVal[nFacts:,nFacts:],eVec[:,nFacts:]
    corr0=np.dot(eVecL,eValL).dot(eVecL.T)
    corr1=np.dot(eVecR,eValR).dot(eVecR.T)
    corr2=corr0+alpha*corr1+(1-alpha)*np.diag(np.diag(corr1))
    return corr2

from sklearn.base import BaseEstimator, TransformerMixin

#@njit
def fillna_npwhere_njit(array, values):
    if np.isnan(array.sum()):
        array = np.where(np.isnan(array), values, array)
    return array

class RMTDenoising(BaseEstimator, TransformerMixin):
    
    def __init__(self, bWidth=.01, alpha=.5, feature_0=True, sample=0.3, seed=2021):
        self.bWidth = bWidth
        self.alpha = alpha
        self.feature_0 = feature_0
        self.sample = sample
        self.seed = seed
    
    def denoise(self, X):
        sample = X.sample(frac=self.sample, random_state=self.seed)
        q = X.shape[0] / X.shape[1]
        cov = sample.cov().values
        corr0 = cov2corr(cov)

        eVal0, eVec0 = getPCA(corr0)
        eMax0, var0 = findMaxEval(np.diag(eVal0), q, bWidth=self.bWidth)
        nFacts0 = eVal0.shape[0] - np.diag(eVal0)[::-1].searchsorted(eMax0)
        corr1 = denoisedCorr2(eVal0,eVec0,nFacts0,alpha=self.alpha)
        eVal1, eVec1 = getPCA(corr1)
        #result = np.hstack((np.diag(eVal1), var0))
        #name = [f'eigen_{i+1}' for i in range(len(eVal1))] + ['var_explained']
        return eVec1[:, :nFacts0]
    
    def fit(self, X, y=None):
        if self.feature_0:
            self.cols_ = [c for c in X.columns if c != 'feature_0']
        else:
            self.cols_ = list(X.columns)
        X_ = X[self.cols_]
        self.W_ = self.denoise(X_)
        self.dim_W_ = self.W_.shape[1]
        return self
    
    def transform(self, X, y=None):
        X_ = X.copy()
        names = [f'proj_{i}' for i in range(self.dim_W_)]
        projection = pd.DataFrame(fillna_npwhere_njit(X_[self.cols_].values, 0).dot(self.W_), columns=names)
        if self.feature_0:
            projection['feature_0'] = X['feature_0']
        return projection

# Configuration 

In [ ]:
SEED = 42 
# INPUT_DIR = '../input/jane-street-market-prediction/'
START_DATE = 85
INPUT_DIR = '../input/janestreet-save-as-feather/'
TRADING_THRESHOLD = 0.502 

# Load data

In [ ]:
os.listdir(INPUT_DIR)

In [ ]:
%%time

def load_data(input_dir=INPUT_DIR):
    train = pd.read_feather(pathlib.Path(input_dir + 'train.feather'))
    #features = pd.read_feather(pathlib.Path(input_dir + 'features.feather'))
    #example_test = pd.read_feather(pathlib.Path(input_dir + 'example_test.feather'))
    #ss = pd.read_feather(pathlib.Path(input_dir + 'example_sample_submission.feather'))
    return train
train = load_data(INPUT_DIR)

### Columns removed in consideration with Feature Selection and Collinearity

In [ ]:
drop = {'feature_100', 'feature_101', 'feature_102', 'feature_107', 'feature_108', 'feature_110','feature_112', 'feature_113', 'feature_114', 'feature_116',
 'feature_119', 'feature_12', 'feature_122', 'feature_124', 'feature_125', 'feature_126', 'feature_127', 'feature_128', 'feature_129', 'feature_14',
 'feature_18', 'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26', 'feature_28', 'feature_30', 'feature_31',
 'feature_32', 'feature_33', 'feature_34', 'feature_35', 'feature_36', 'feature_38', 'feature_4', 'feature_40', 'feature_47', 'feature_48',
 'feature_49', 'feature_51', 'feature_57', 'feature_58', 'feature_6', 'feature_61', 'feature_63', 'feature_66', 'feature_68', 'feature_69', 'feature_71', 'feature_76',
 'feature_8', 'feature_88', 'feature_96'}

In [ ]:
# reduce train
train = train.query(f'date > {START_DATE}')
train.fillna(train.mean(),inplace=True)
train = train[train['weight'] != 0]
train = train.drop(drop,axis=1)

In [ ]:
targets = ['resp','resp_1','resp_2','resp_3','resp_4']

targets_f0 = targets + ['feature_0']
target_tf = RMTDenoising(sample=0.8)

target_tf.fit(train[targets_f0])

Dn_targets = target_tf.transform(train[targets_f0])

train['dresp'] = -Dn_targets.proj_0

#train['action'] =  (  (train['resp_1'] > 0 ) & (train['resp_2'] > 0 ) & (train['resp_3'] > 0 ) & (train['resp_4'] > 0 ) &  (train['resp'] > 0 ) &  (train['dresp'] > 0 )   ).astype('int')

# Model

In [ ]:
# feats
features = train.columns[train.columns.str.startswith('feature')].values.tolist()
print('{} features used'.format(len(features)))

In [ ]:
# target
train['action'] = (train['resp'] > 0).astype('int')
f_mean = np.mean(train[features[1:]].values,axis=0)

### Params - [Got through using Optuna in google colab notebooks]

In [ ]:
params = {'n_estimators': 658, 'max_depth': 5, 'min_child_weight': 5, 'learning_rate': 0.04067234688762774,
 'subsample': 0.8550706590979082, 'gamma': 0, 'colsample_bytree': 0.976077957177498,
'objective':'binary:logistic','eval_metric': 'auc','tree_method': 'gpu_hist',  
        'random_state': 42,} # best 5600
params_1 = {'n_estimators': 819, 'max_depth': 6, 'min_child_weight': 3, 'learning_rate':0.02394470312114685,
 'subsample':  0.7303852815955237, 'gamma': 0, 'colsample_bytree':  0.770496590789439,
'objective':'binary:logistic','eval_metric': 'auc','tree_method': 'gpu_hist',  
        'random_state': 42,} # 5000

params_2 = {'n_estimators': 752, 'max_depth': 11, 'min_child_weight': 4, 'learning_rate':0.03586904895157962,
 'subsample':  0.6299777543887805, 'gamma': 0, 'colsample_bytree':  0.8066226408240955,
'objective':'binary:logistic','eval_metric': 'auc','tree_method': 'gpu_hist',  
        'random_state': 42,} # 3000

params_3 = {'n_estimators': 766, 'max_depth': 10, 'min_child_weight': 7, 'learning_rate': 0.010029649857344392,
 'subsample':  0.8377963223123193, 'gamma': 0, 'colsample_bytree':  0.965897457878852,
'objective':'binary:logistic','eval_metric': 'auc','tree_method': 'gpu_hist',  
        'random_state': 42,}

params_4 =  {'n_estimators': 991, 'max_depth': 7, 'min_child_weight': 2,
  'learning_rate': 0.021441916265932923, 'subsample': 0.6508037151907303, 
  'gamma': 0, 'colsample_bytree': 0.8545754382982661,'objective':'binary:logistic',
'eval_metric': 'auc','tree_method': 'gpu_hist', 'random_state': 42,}

# Training 5 Models 

In [ ]:
training = True
if training:
    import time
    import gc
    resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']
    X = train[features].values
    #y = train['action'].values
    y = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T #Multitarget
    groups = train['date'].values
    models = []
    scores = []

    cv = PurgedGroupTimeSeriesSplit(
        n_splits=4,
        group_gap=20,
    )
    for t in tqdm(range(y.shape[1])):
        yy = y[:,t]
        for i, (train_index, valid_index) in enumerate(cv.split(
                X,
                yy,
                groups=groups)):
            print(f'Target {t} Fold {i} started at {time.ctime()}')
            X_train, X_valid = X[train_index], X[valid_index]
            y_train, y_valid = yy[train_index], yy[valid_index]
            model = xgb.XGBClassifier(**params_4, n_jobs = -1)
            model.fit(X_train, y_train, 
                    eval_set=[(X_valid, y_valid)], eval_metric='auc',
                    verbose=100, callbacks = [xgb.callback.EarlyStopping(rounds=300,save_best=True)])
            pred = model.predict(X_valid)
            score = roc_auc_score(y_valid,pred)
            model.save_model(f'my_model_{t}_{i}.model')
            models.append(model)
            scores.append(score)
            del score, model
        print(scores)
        del X_train, X_valid, y_train, y_valid
        rubbish = gc.collect()

else:
    predictor_0 = treelite_runtime.Predictor('../input/xgbtreelite000/mymodel_0.so', verbose=True)
    predictor_1 = treelite_runtime.Predictor('../input/xgbtreelite000/mymodel_1.so', verbose=True)
    predictor_2 = treelite_runtime.Predictor('../input/xgbtreelite000/mymodel_2.so', verbose=True)
    predictor_3 = treelite_runtime.Predictor('../input/xgbtreelite000/mymodel_3.so', verbose=True)
    predictor_4 = treelite_runtime.Predictor('../input/xgbtreelite000/mymodel_4.so', verbose=True)

# Compile with Treelite
Simply follow the tutorial: https://treelite.readthedocs.io/en/latest/tutorials/first.html

In [ ]:
# pass to treelite
if training:
    model_0 = treelite.Model.load('my_model_0_3.model', model_format='xgboost')
    model_1 = treelite.Model.load('my_model_1_3.model', model_format='xgboost')
    model_2 = treelite.Model.load('my_model_2_3.model', model_format='xgboost')
    model_3 = treelite.Model.load('my_model_3_3.model', model_format='xgboost')
    model_4 = treelite.Model.load('my_model_4_3.model', model_format='xgboost')
    #model_5 = treelite.Model.load('my_model_5_3.model', model_format='xgboost')

In [ ]:

if training:
    m = [model_0,model_1,model_2,model_3,model_4]
    for j,i in enumerate(m):
        toolchain = 'gcc'
        i.export_lib(toolchain=toolchain, libpath=f'./mymodel_{j}.so',
                     params={'parallel_comp': 32}, verbose=True)

In [ ]:
# predictor from treelite
if training:
    predictor_0 = treelite_runtime.Predictor(f'./mymodel_{0}.so', verbose=True)
    predictor_1 = treelite_runtime.Predictor(f'./mymodel_{1}.so', verbose=True)
    predictor_2 = treelite_runtime.Predictor(f'./mymodel_{2}.so', verbose=True)
    predictor_3 = treelite_runtime.Predictor(f'./mymodel_{3}.so', verbose=True)
    predictor_4 = treelite_runtime.Predictor(f'./mymodel_{4}.so', verbose=True)

# Submit

In [ ]:
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

In [ ]:
f = np.median 
index_features = [n for n in range(1,(len(features) + 1))]
for (test_df, pred_df) in tqdm(iter_test):
    
    if test_df['weight'].item() > 0:
        
        test_df = test_df.drop(drop,axis=1)
        x_tt = test_df.values[0][index_features].reshape(1,-1)
            
        if np.isnan(x_tt[:, 1:].sum()):
            x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
        
        # inference with treelite
        batch = treelite_runtime.Batch.from_npy2d(x_tt)
        pred_0 = predictor_0.predict(batch)
        pred_1 = predictor_1.predict(batch)
        pred_2 = predictor_2.predict(batch)
        pred_3 = predictor_3.predict(batch)
        pred_4 = predictor_4.predict(batch)
        
        # Version 9
        pred = np.stack([pred_0,pred_1,pred_2,pred_3,pred_4],axis=0).T
        pred = f(pred)
        pred_df.action = int(pred >= TRADING_THRESHOLD)

    else:
         pred_df['action'].values[0] = 0
    env.predict(pred_df)